In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')

import numpy as np

from minefield import Minefield
from network import DQ_Sweeper

from tqdm import tqdm

rewards = {'safe_reward' : 15, 
          'repeat_loss' : -5, 
          'mine_loss' : -10, 
          'completion_reward' : 30}
env = Minefield(8, 8, 10, **rewards)

filters = [10, 8, 5]
sizes = [5, 5, 5]
env_shape = (8, 8, 11)

model = DQ_Sweeper(env, filters, sizes, 0.9, 0.9995, 10, 0.2)

In [ ]:
model.training_loop(100000, 128)

 10%|▉         | 9998/100000 [20:56<2:32:39,  9.83it/s] 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: Models/model_10000_49_12_15Jun21/assets


 20%|█▉        | 19999/100000 [47:27<3:20:30,  6.65it/s] 

INFO:tensorflow:Assets written to: Models/model_20000_15_13_15Jun21/assets


 29%|██▉       | 29273/100000 [1:20:05<6:15:35,  3.14it/s] 

In [ ]:
plt.figure(figsize = (20, 10))
sns.regplot(x = np.arange(len(model.logger.tests)), y = model.logger.tests)

In [ ]:
model.show_single('hundok')

In [ ]:
env3 = Minefield(10, 10, 30, **rewards)

model3 = DQ_Sweeper(env3, filters, sizes, 0.01, 1, 100, 0.2, weights = 'model_June09')

model3.show_single('bigger_hundok')

In [ ]:
0.9995**10000